# 01 LanceDB 介绍与基础操作

本 Notebook 介绍向量数据库的概念，以及 LanceDB 的基础 CRUD 操作。

**本节无需 API Key**，使用手动构造的向量来演示核心操作。

## 1. 什么是向量数据库？

传统数据库通过精确匹配（如 SQL WHERE 条件）来查询数据。而向量数据库存储的是高维向量（如 1024 维浮点数组），通过计算向量之间的**距离**来查找最相似的数据。

核心应用场景：
- **语义搜索**：根据含义而非关键词搜索（"好用" 能匹配到 "质量不错"）
- **推荐系统**：找到与用户偏好相似的商品
- **RAG（检索增强生成）**：为大语言模型提供相关上下文

### LanceDB 的定位

LanceDB 是一个**嵌入式**向量数据库，底层使用 Lance 列式存储格式：

- 无需部署独立服务器，`pip install` 即可使用
- 支持向量搜索 + 标量过滤的混合查询
- 底层 Lance 格式可被 Daft、DuckDB 等引擎直接读取
- 零拷贝访问，内存效率高

## 2. 安装验证

In [ ]:
import lancedb

print(f"LanceDB version: {lancedb.__version__}")

## 3. 连接数据库

LanceDB 使用本地目录作为数据库存储路径，连接时如果目录不存在会自动创建。

In [ ]:
db = lancedb.connect("../lancedb_data")

print(f"数据库路径: ../lancedb_data")
print(f"已有表: {db.table_names()}")

## 4. CRUD 操作

下面用手动构造的 3 维向量来演示 LanceDB 的核心操作。

在实际应用中，向量通常是由嵌入模型（如 Qwen3-Embedding）生成的 1024 维浮点数组，这里用 3 维向量是为了便于理解。

### 4.1 创建表（create_table）

创建表时需要提供初始数据，LanceDB 会自动推断 schema。`vector` 列会被识别为向量列。

In [ ]:
data = [
    {"id": 1, "text": "苹果很甜", "category": "水果", "vector": [1.0, 0.0, 0.0]},
    {"id": 2, "text": "香蕉很软", "category": "水果", "vector": [0.9, 0.1, 0.0]},
    {"id": 3, "text": "手机很快", "category": "电子", "vector": [0.0, 1.0, 0.0]},
    {"id": 4, "text": "电脑很强", "category": "电子", "vector": [0.0, 0.9, 0.1]},
    {"id": 5, "text": "衣服很暖", "category": "服装", "vector": [0.0, 0.0, 1.0]},
]

# mode="overwrite" 表示如果表已存在则覆盖
table = db.create_table("demo_items", data, mode="overwrite")

print(f"表名: {table.name}")
print(f"行数: {table.count_rows()}")
print(f"Schema: {table.schema}")

### 4.2 向量搜索（search）

向量搜索的核心是：给定一个查询向量，找到表中距离最近的 N 条记录。

LanceDB 默认使用 **L2 距离**（欧氏距离），`_distance` 越小表示越相似。常见的距离度量还有：
- **cosine**：余弦距离，适合文本嵌入
- **dot**：点积距离

In [ ]:
# 查询向量接近 "水果" 类（[1, 0, 0] 方向）
query_vector: list[float] = [0.95, 0.05, 0.0]

results = table.search(query_vector).limit(3).to_pandas()
print("查询: 接近 '水果' 方向的向量")
print(results[["id", "text", "category", "_distance"]])

In [ ]:
# 查询向量接近 "电子" 类（[0, 1, 0] 方向）
query_vector: list[float] = [0.0, 0.95, 0.05]

results = table.search(query_vector).limit(3).to_pandas()
print("查询: 接近 '电子' 方向的向量")
print(results[["id", "text", "category", "_distance"]])

### 4.3 插入数据（add）

向已有表中追加新数据。

In [ ]:
new_data = [
    {"id": 6, "text": "橙子很酸", "category": "水果", "vector": [0.8, 0.2, 0.0]},
    {"id": 7, "text": "耳机很棒", "category": "电子", "vector": [0.1, 0.85, 0.05]},
]

table.add(new_data)
print(f"插入后行数: {table.count_rows()}")

In [ ]:
# 验证新数据可被搜索到
results = table.search([0.85, 0.15, 0.0]).limit(3).to_pandas()
print(results[["id", "text", "category", "_distance"]])

### 4.4 删除数据（delete）

通过 SQL 风格的过滤条件删除数据。

In [ ]:
print(f"删除前行数: {table.count_rows()}")

# 删除 id=6 的记录
table.delete("id = 6")
print(f"删除 id=6 后行数: {table.count_rows()}")

# 也可以按条件批量删除
table.delete("category = '服装'")
print(f"删除服装类后行数: {table.count_rows()}")

### 4.5 更新数据

LanceDB 通过**删除 + 插入**实现更新操作。

In [ ]:
# 更新 id=1 的记录：修改文本和向量
table.delete("id = 1")
table.add([{"id": 1, "text": "苹果又甜又脆", "category": "水果", "vector": [0.95, 0.0, 0.05]}])

# 验证更新结果
results = table.search([0.95, 0.0, 0.05]).limit(1).to_pandas()
print("更新后的记录:")
print(results[["id", "text", "category"]])

### 4.6 混合查询：向量搜索 + 标量过滤

LanceDB 支持在向量搜索的同时添加 SQL 风格的过滤条件，这在实际应用中非常常用。

In [ ]:
# 先恢复完整数据
full_data = [
    {"id": 1, "text": "苹果很甜", "category": "水果", "vector": [1.0, 0.0, 0.0]},
    {"id": 2, "text": "香蕉很软", "category": "水果", "vector": [0.9, 0.1, 0.0]},
    {"id": 3, "text": "手机很快", "category": "电子", "vector": [0.0, 1.0, 0.0]},
    {"id": 4, "text": "电脑很强", "category": "电子", "vector": [0.0, 0.9, 0.1]},
    {"id": 5, "text": "衣服很暖", "category": "服装", "vector": [0.0, 0.0, 1.0]},
]
table = db.create_table("demo_items", full_data, mode="overwrite")

# 搜索最相似的向量，但只返回 "水果" 类
query_vector: list[float] = [0.5, 0.5, 0.0]
results = (
    table
    .search(query_vector)
    .where("category = '水果'")
    .limit(3)
    .to_pandas()
)
print("混合查询: 向量搜索 + 只看水果类")
print(results[["id", "text", "category", "_distance"]])

## 5. 预览评论数据集

在下一个 Notebook 中，我们将使用真实的中文商品评论数据集来演示嵌入生成和语义搜索。先来预览一下数据。

> 如果还没有准备数据，请先运行: `python demo3_lancedb/data/prepare_data.py --output demo3_lancedb/data`

In [ ]:
import pandas as pd

reviews = pd.read_csv("../data/reviews.csv")
print(f"数据集大小: {len(reviews):,} 条")
print(f"列: {list(reviews.columns)}")
print(f"\n类别分布:")
print(reviews["cat"].value_counts())
print(f"\n标签分布 (1=正面, 0=负面):")
print(reviews["label"].value_counts())

In [ ]:
# 查看几条示例评论
reviews.sample(5, random_state=42)[["cat", "label", "review"]]

## 6. 清理

删除本 Notebook 创建的演示表。

In [ ]:
db.drop_table("demo_items")
print(f"剩余表: {db.table_names()}")

## 小结

本 Notebook 介绍了：
- 向量数据库的概念：通过向量距离查找相似数据
- LanceDB 的定位：嵌入式、零部署、Lance 列式存储
- CRUD 操作：`create_table`、`search`、`add`、`delete`
- 混合查询：向量搜索 + 标量过滤

下一个 Notebook 将使用嵌入模型将评论文本转换为向量，实现真正的语义搜索。